In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import glob
import cartopy as cart
import cartopy.crs as ccrs
notebook_path = os.getcwd()

In [3]:
# setting up a dask LocalCluster
from dask.distributed import Client
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

ImportError: cannot import name 'validate_key' from 'dask.core' (C:\Users\aline\mambaforge\envs\gtc\Lib\site-packages\dask\core.py)

In [4]:
## Import the ecco_v4_py library into Python
## =========================================
##    If ecco_v4_py is not installed in your local Python library,
##    tell Python where to find it.  The example below adds
##    ecco_v4_py to the user's path if it is stored in the folder
##    ECCOv4-py under the user's home directory

from os.path import join,expanduser,exists,split
user_home_dir = expanduser('~')

sys.path.append(join(user_home_dir,'ECCOv4-py'))

import ecco_v4_py as ecco

In [28]:
DIR = "H:/.shortcut-targets-by-id/1wvJjD0RMTujKYaXQapEiGk-Mx03_KSin/GTC/"

parameter = "STRESS" # OBP - SSH - TEMP_SALINITY - STRESS
DATA_DIR = DIR + f"solodoch_data_full/26N/ECCO_L4_{parameter}_05DEG_MONTHLY_V4R4"

In [29]:
files = [f for f in os.listdir(DATA_DIR) if os.path.isfile(os.path.join(DATA_DIR, f))]
first_file_path = os.path.join(DATA_DIR, files[0])
first_file_path

'H:/.shortcut-targets-by-id/1wvJjD0RMTujKYaXQapEiGk-Mx03_KSin/GTC/solodoch_data_full/26N/ECCO_L4_STRESS_05DEG_MONTHLY_V4R4\\OCEAN_AND_ICE_SURFACE_STRESS_mon_mean_1992-06_ECCO_V4r4_latlon_0p50deg.nc'

In [30]:
ecco_grid = xr.open_dataset(first_file_path)
ecco_grid

<xarray.Dataset>
Dimensions:         (time: 1, latitude: 1, longitude: 720, nv: 2)
Coordinates:
    latitude_bnds   (latitude, nv) float32 ...
    time_bnds       (time, nv) datetime64[ns] ...
    longitude_bnds  (longitude, nv) float32 ...
  * latitude        (latitude) float32 26.25
  * longitude       (longitude) float32 -179.8 -179.2 -178.8 ... 179.2 179.8
  * nv              (nv) float32 0.0 1.0
  * time            (time) datetime64[ns] 1992-06-16
Data variables:
    oceTAUE         (time, latitude, longitude) float32 ...
    EXFtaun         (time, latitude, longitude) float32 ...
    EXFtaue         (time, latitude, longitude) float32 ...
    oceTAUN         (time, latitude, longitude) float32 ...
Attributes: (12/58)
    acknowledgement:              This research was carried out by the Jet Pr...
    author:                       Ian Fenty and Ou Wang
    cdm_data_type:                Grid
    comment:                      Fields provided on a regular lat-lon grid. ...
    Conventions:                  CF-1.8, ACDD-1.3
    coordinates_comment:          Note: the global 'coordinates' attribute de...
    ...                           ...
    time_coverage_end:            1992-07-01T00:00:00
    time_coverage_resolution:     P1M
    time_coverage_start:          1992-06-01T00:00:00
    title:                        ECCO Ocean and Sea-Ice Surface Stress - Mon...
    uuid:                         dcf29cda-4158-11eb-a17e-0cc47a3f68cb
    history_json:                 [{"$schema":"https:\/\/harmony.earthdata.na...

In [31]:
## Set top-level file directory for the ECCO NetCDF files
## =================================================================

## currently set to ~/Downloads/ECCO_V4r4_PODAAC,
## the default if ecco_podaac_download was used to download dataset granules
ECCO_dir = join(user_home_dir,'Downloads','ECCO_V4r4_PODAAC')

In [32]:
## Load the model grid
ecco_grid = xr.open_dataset(glob.glob(join(ECCO_dir,'*GEOMETRY*','*.nc'))[0])

## Load ECCO variables
ecco_vars_adv_th = xr.open_mfdataset(join(ECCO_dir,'*_OCEAN_3D_TEMPERATURE_FLUX*MONTHLY*','*.nc'),\
                                      data_vars='minimal',coords='minimal',compat='override')
ecco_vars_uvw = xr.open_mfdataset(join(ECCO_dir,'*_OCEAN_3D_VOLUME_FLUX*MONTHLY*','*.nc'),\
                                      data_vars='minimal',coords='minimal',compat='override')

ecco_vars = xr.merge((ecco_vars_adv_th[['ADVx_TH','ADVy_TH']],ecco_vars_uvw[['UVELMASS','VVELMASS']]))

## Merge the ecco_grid with the ecco_vars to make the ecco_ds
ds = xr.merge((ecco_grid , ecco_vars))